# 01_mm_all_improved.ipynb – Optimized Multimodal Price Prediction Pipeline

# Cell 1: Setup (Environment, Device, Paths, Seeding)

In [1]:
import os
import sys
import json
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Add project root to Python path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Local helpers
from src.utils.seed import seed_everything
from src.training.metrics import smape_np
from src.data.load import read_csvs, make_folds

# Load environment
load_dotenv()

SEED = 42
DEVICE = torch.device(os.getenv("DEVICE", "mps"))
TRAIN_CSV = str(project_root / os.getenv("TRAIN_CSV", "dataset/train.csv"))
TEST_CSV = str(project_root / os.getenv("TEST_CSV", "dataset/test.csv"))
IMG_DIR = project_root / Path(os.getenv("IMG_DIR", "data/processed/images"))
OUT_DIR = project_root / Path(os.getenv("OUT_DIR", "outputs"))
HF_HOME = project_root / Path(os.getenv("HF_HOME", ".hf_cache"))

for p in [IMG_DIR, OUT_DIR, HF_HOME, OUT_DIR/"oof", OUT_DIR/"test_preds", OUT_DIR/"reports"]:
    p.mkdir(parents=True, exist_ok=True)

seed_everything(SEED)

print("Torch:", torch.__version__)
print("MPS built:", torch.backends.mps.is_built())
print("MPS available:", torch.backends.mps.is_available())
print("Device:", DEVICE)
print("Train CSV:", TRAIN_CSV)
print("Test  CSV:", TEST_CSV)

Torch: 2.8.0
MPS built: True
MPS available: True
Device: mps
Train CSV: /Users/adyasrivastava/Downloads/smart_pricing/dataset/train.csv
Test  CSV: /Users/adyasrivastava/Downloads/smart_pricing/dataset/test.csv


# Cell 2: Load Data + Feature Engineering

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit

# Read data
train_df, test_df = read_csvs(TRAIN_CSV, TEST_CSV)
train_df = make_folds(train_df, n_folds=5, seed=SEED)

# Create holdout split
y_log = np.log1p(train_df["price"].clip(lower=0.0))
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=SEED)
holdout_tr_idx, holdout_va_idx = next(sss.split(np.zeros(len(y_log)), pd.qcut(y_log, q=20, duplicates="drop")))
hold_tr_df = train_df.iloc[holdout_tr_idx].copy()
hold_va_df = train_df.iloc[holdout_va_idx].copy()

print("\n" + "="*60)
print("FEATURE ENGINEERING")
print("="*60)

def extract_features(df):
    """Extract price-relevant features from catalog content"""
    df = df.copy()
    text = df['catalog_content'].fillna('')
    
    # Package indicators
    df['has_pack'] = text.str.contains(r'\bpack\b|\bcount\b|\bct\b', case=False, regex=True).astype(int)
    df['has_multipack'] = text.str.contains(r'\d+\s*pack', case=False, regex=True).astype(int)
    
    # Weight/volume indicators
    df['has_oz'] = text.str.contains(r'\boz\b|ounce', case=False, regex=True).astype(int)
    df['has_lb'] = text.str.contains(r'\blb\b|pound', case=False, regex=True).astype(int)
    df['has_kg'] = text.str.contains(r'\bkg\b|kilogram', case=False, regex=True).astype(int)
    df['has_ml'] = text.str.contains(r'\bml\b|milliliter', case=False, regex=True).astype(int)
    df['has_l'] = text.str.contains(r'\bl\b|liter', case=False, regex=True).astype(int)
    
    # Extract numeric values
    def extract_numbers(s):
        nums = re.findall(r'\d+\.?\d*', str(s))
        return [float(n) for n in nums]
    
    df['num_count'] = text.apply(lambda x: len(extract_numbers(x)))
    df['max_number'] = text.apply(lambda x: max(extract_numbers(x), default=0))
    df['min_number'] = text.apply(lambda x: min(extract_numbers(x), default=0))
    df['sum_numbers'] = text.apply(lambda x: sum(extract_numbers(x)))
    
    # Text statistics
    df['text_len'] = text.str.len()
    df['word_count'] = text.str.split().str.len()
    df['avg_word_len'] = text.apply(lambda x: np.mean([len(w) for w in str(x).split()]) if x else 0)
    df['num_digits'] = text.apply(lambda x: sum(c.isdigit() for c in str(x)))
    
    # Brand/quality indicators
    premium_brands = ['organic', 'premium', 'gourmet', 'artisan', 'natural', 'prime']
    budget_brands = ['value', 'budget', 'economy', 'basic']
    
    df['is_premium'] = text.str.lower().str.contains('|'.join(premium_brands), regex=True).astype(int)
    df['is_budget'] = text.str.lower().str.contains('|'.join(budget_brands), regex=True).astype(int)
    
    # Category indicators (food vs non-food)
    food_keywords = ['food', 'snack', 'drink', 'beverage', 'grocery', 'eat']
    df['is_food'] = text.str.lower().str.contains('|'.join(food_keywords), regex=True).astype(int)
    
    # Size indicators
    size_keywords = {'small': 1, 'medium': 2, 'large': 3, 'xl': 4, 'jumbo': 5}
    df['size_indicator'] = 0
    for keyword, value in size_keywords.items():
        df.loc[text.str.lower().str.contains(keyword, regex=False), 'size_indicator'] = value
    
    return df

print("Extracting features from train set...")
train_df = extract_features(train_df)
print("Extracting features from test set...")
test_df = extract_features(test_df)
print("Extracting features from holdout sets...")
hold_tr_df = extract_features(hold_tr_df)
hold_va_df = extract_features(hold_va_df)

# Feature columns (only include features that actually exist)
feature_cols = [
    'has_pack','has_multipack','has_oz','has_lb','has_kg','has_ml','has_l',
    'num_count','max_number','min_number','sum_numbers',
    'text_len','word_count','avg_word_len','num_digits',
    'is_premium','is_budget','is_food','size_indicator',
    'has_image'  # This comes from read_csvs() function
]

print(f"\nExtracted {len(feature_cols)} features:")
print(feature_cols)

# Standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_feats = scaler.fit_transform(train_df[feature_cols])
test_feats = scaler.transform(test_df[feature_cols])
holdtr_feats = scaler.transform(hold_tr_df[feature_cols])
holdva_feats = scaler.transform(hold_va_df[feature_cols])

print(f"\nFeature shapes:")
print(f"  Train: {train_feats.shape}")
print(f"  Test: {test_feats.shape}")
print("="*60)

print(f"\nData shapes:")
print(f"  Train: {len(train_df)} | Test: {len(test_df)}")
print(f"  Holdout train: {len(hold_tr_df)} | Holdout val: {len(hold_va_df)}")

Loading training data from: /Users/adyasrivastava/Downloads/smart_pricing/dataset/train.csv
Loading test data from: /Users/adyasrivastava/Downloads/smart_pricing/dataset/test.csv

Train shape: (75000, 4)
Test shape: (75000, 3)

Price statistics:
  Min: $0.13
  Max: $2796.00
  Mean: $23.65
  Median: $14.00

Images available:
  Train: 75000/75000 (100.0%)
  Test: 75000/75000 (100.0%)

Created 5 stratified folds:
  Fold 0: 15000 samples, mean price: $23.46, median: $14.10
  Fold 1: 15000 samples, mean price: $23.63, median: $13.99
  Fold 2: 15000 samples, mean price: $23.47, median: $14.20
  Fold 3: 15000 samples, mean price: $23.96, median: $13.99
  Fold 4: 15000 samples, mean price: $23.73, median: $14.05

FEATURE ENGINEERING
Extracting features from train set...
Extracting features from test set...
Extracting features from holdout sets...

Extracted 20 features:
['has_pack', 'has_multipack', 'has_oz', 'has_lb', 'has_kg', 'has_ml', 'has_l', 'num_count', 'max_number', 'min_number', 'sum_

In [3]:
# ---- NEW CELL: Quantity & Unit Normalization ----
import re

_QTY_PAT = re.compile(r'(?P<num>\d+(?:\.\d+)?)\s*(?P<u>kg|g|gram|grams|lb|lbs|pound|pounds|oz|ounce|ounces|l|liter|litre|ml|milliliter|millilitre)\b', re.I)
_PACK_PAT = re.compile(r'(?:(?P<count>\d+)\s*[x×*]\s*)?(?P<size>\d+(?:\.\d+)?)\s*(?P<u>kg|g|gram|grams|lb|lbs|pound|pounds|oz|ounce|ounces|l|liter|litre|ml|milliliter|millilitre)\b', re.I)
_SINGLE_PACK = re.compile(r'\b(\d+)\s*pack\b|\bpack\s*of\s*(\d+)\b|\b(\d+)\s*ct\b|\b(\d+)\s*count\b', re.I)

def _to_grams(num, u):
    u = u.lower()
    if u in ["kg"]:   return num * 1000
    if u in ["g","gram","grams"]: return num
    if u in ["lb","lbs","pound","pounds"]: return num * 453.592
    if u in ["oz","ounce","ounces"]: return num * 28.3495
    return None

def _to_milliliters(num, u):
    u = u.lower()
    if u in ["l","liter","litre"]: return num * 1000
    if u in ["ml","milliliter","millilitre"]: return num
    # some products label oz fluid but ambiguous; skip to stay safe
    return None

def parse_qty_features(s):
    s = str(s)
    grams_list, ml_list = [], []
    for m in _QTY_PAT.finditer(s):
        val = float(m.group("num"))
        u   = m.group("u")
        g = _to_grams(val, u); ml = _to_milliliters(val, u)
        if g is not None: grams_list.append(g)
        if ml is not None: ml_list.append(ml)

    # pack patterns like "3 x 200 g"
    pack_count = 1
    for m in _PACK_PAT.finditer(s):
        count = m.group("count")
        if count: pack_count = max(pack_count, int(count))

    # patterns like "pack of 6" or "6 pack"
    for m in _SINGLE_PACK.finditer(s):
        nums = [int(x) for x in m.groups() if x]
        if nums: pack_count = max(pack_count, max(nums))

    return {
        "content_mass_g": np.max(grams_list) if grams_list else 0.0,
        "content_vol_ml": np.max(ml_list) if ml_list else 0.0,
        "pack_count": float(pack_count),
        "has_qty": int(bool(grams_list or ml_list)),
    }

def add_qty_columns(df):
    feats = df["catalog_content"].fillna("").apply(parse_qty_features).apply(pd.Series)
    return pd.concat([df, feats], axis=1)

# apply to all splits
train_df = add_qty_columns(train_df)
test_df  = add_qty_columns(test_df)
hold_tr_df = add_qty_columns(hold_tr_df)
hold_va_df = add_qty_columns(hold_va_df)

# Heuristic: estimated number of units (mass or volume) to stabilize scale
for d in (train_df, test_df, hold_tr_df, hold_va_df):
    d["est_units"] = np.where(d["content_mass_g"]>0, d["content_mass_g"]/100.0,
                         np.where(d["content_vol_ml"]>0, d["content_vol_ml"]/100.0, 0.0))

# Cell 3: Map Image Paths

In [4]:
from urllib.parse import urlparse

print(f"Image directory: {IMG_DIR}")

def to_local_path(url: str) -> str:
    p = urlparse(str(url))
    name = os.path.basename(p.path) or "na.jpg"
    return (IMG_DIR / name).as_posix()

for df in (train_df, test_df, hold_tr_df, hold_va_df):
    df["image_path"] = df["image_link"].fillna("").map(to_local_path)

# Check existing images
print("Checking existing images...")
train_exists = sum([os.path.exists(p) for p in train_df["image_path"].tolist()])
test_exists = sum([os.path.exists(p) for p in test_df["image_path"].tolist()])

print(f"\n📊 Images Available:")
print(f"   Train: {train_exists:,}/{len(train_df):,} ({100*train_exists/len(train_df):.1f}%)")
print(f"   Test:  {test_exists:,}/{len(test_df):,} ({100*test_exists/len(test_df):.1f}%)")
print(f"\n✅ Proceeding with available images")

Image directory: /Users/adyasrivastava/Downloads/smart_pricing/data/processed/images
Checking existing images...

📊 Images Available:
   Train: 53,988/75,000 (72.0%)
   Test:  3,787/75,000 (5.0%)

✅ Proceeding with available images


# Cell 4: MiniLM Text Embeddings + Feature Concatenation

In [ ]:
# ============================================================
# Cell 4 (REPLACED): e5-small-v2 Text Embeddings with Attention Pooling
# ============================================================
from transformers import AutoTokenizer, AutoModel
from tqdm import trange

# --- Swap the encoder here ---
MINILM = "intfloat/e5-small-v2"   # (Apache-2.0). Good for product-ish text.
# If you want to try BGE instead:
# MINILM = "BAAI/bge-small-en-v1.5"  # (MIT)  NOTE: set use_e5_prefix=False below for BGE.

tok_minilm = AutoTokenizer.from_pretrained(MINILM, cache_dir=HF_HOME.as_posix())
enc_minilm = AutoModel.from_pretrained(MINILM, cache_dir=HF_HOME.as_posix()).to(DEVICE)
enc_minilm.eval()

# --- Attention pooling (replaces mean pooling) ---
class AttnPool(nn.Module):
    def __init__(self, hidden: int):
        super().__init__()
        self.proj = nn.Linear(hidden, 1, bias=False)

    def forward(self, hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        # hidden_states: [B, T, H], attention_mask: [B, T] (1 for tokens, 0 for pad)
        scores = self.proj(hidden_states).squeeze(-1)                 # [B, T]
        scores = scores.masked_fill(attention_mask == 0, -1e9)        # mask pads
        weights = torch.softmax(scores, dim=1).unsqueeze(-1)          # [B, T, 1]
        pooled = (hidden_states * weights).sum(dim=1)                 # [B, H]
        return pooled

attn_pool = AttnPool(enc_minilm.config.hidden_size).to(DEVICE)

@torch.no_grad()
def encode_minilm(texts, batch_size=128, max_len=256, cache_path=None, force_recompute=False, use_e5_prefix=True):
    """
    Encode texts to L2-normalized sentence embeddings using:
      - Frozen encoder (e5-small-v2 by default)
      - Attention pooling over tokens
      - MPS-friendly float32 output

    Args:
        use_e5_prefix: for e5 models, prepends "query: " to inputs (recommended).
                       If you switch to BGE, set this False.
    """
    cache_path = Path(cache_path) if cache_path is not None else None
    if cache_path and cache_path.exists() and not force_recompute:
        return np.load(cache_path)

    out = []
    desc = "e5-small+attn @ MPS" if use_e5_prefix else "text+attn @ MPS"
    for i in trange(0, len(texts), batch_size, desc=desc):
        batch = texts[i:i+batch_size]
        if use_e5_prefix:
            # e5 models were trained with task prefixes; "query: " is standard
            batch = [f"query: {b}" for b in batch]

        enc = tok_minilm(
            batch,
            padding=True,
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        )
        enc = {k: v.to(DEVICE) for k, v in enc.items()}

        outputs = enc_minilm(**enc)               # last_hidden_state: [B, T, H]
        last = outputs.last_hidden_state
        pooled = attn_pool(last, enc["attention_mask"])              # [B, H]
        pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)   # L2 norm
        out.append(pooled.to(torch.float32).cpu().numpy())

    arr = np.vstack(out).astype(np.float32)
    if cache_path:
        np.save(cache_path, arr)
    return arr

# ---------------------------
# Compute & cache embeddings
# ---------------------------
CACHE_TXT = Path("data/processed/improved_text_attn_e5")
CACHE_TXT.mkdir(parents=True, exist_ok=True)

train_texts  = train_df["catalog_content"].fillna("").astype(str).tolist()
test_texts   = test_df["catalog_content"].fillna("").astype(str).tolist()
holdtr_texts = hold_tr_df["catalog_content"].fillna("").astype(str).tolist()
holdva_texts = hold_va_df["catalog_content"].fillna("").astype(str).tolist()

print("Encoding text with e5-small-v2 + Attention Pooling...")
Etxt_train_emb  = encode_minilm(train_texts,  cache_path=CACHE_TXT/"e5_train_attn.npy",  use_e5_prefix=True)
Etxt_test_emb   = encode_minilm(test_texts,   cache_path=CACHE_TXT/"e5_test_attn.npy",   use_e5_prefix=True)
Etxt_holdtr_emb = encode_minilm(holdtr_texts, cache_path=CACHE_TXT/"e5_holdtr_attn.npy", use_e5_prefix=True)
Etxt_holdva_emb = encode_minilm(holdva_texts, cache_path=CACHE_TXT/"e5_holdva_attn.npy", use_e5_prefix=True)

print("Shapes — text embeddings (attn pooled, e5):")
print("  train:", Etxt_train_emb.shape, " test:", Etxt_test_emb.shape)
print("  hold_tr:", Etxt_holdtr_emb.shape, " hold_va:", Etxt_holdva_emb.shape)

# IMPORTANT:
# Do NOT concatenate handcrafted features here anymore.
# You'll concatenate inside the fold loops (Cell 7 & 9) after fold-local scaling.

Encoding text with e5-small-v2 + Attention Pooling...


e5-small+attn @ MPS:  78%|███████▊  | 459/586 [30:06<08:10,  3.86s/it]

# Cell 5: CLIP Embeddings (Text + Image)

In [ ]:
# ============================================================
# Cell 5 (REPLACED): CLIP Embeddings (Text + Image with TTA)
# - Text: CLIP text features (no change)
# - Image: TTA = average(original, horizontal flip)
# ============================================================

from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from tqdm import tqdm, trange  # trange used in text emb cell as well

CLIP_NAME = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(CLIP_NAME, cache_dir=HF_HOME.as_posix())
clip_model = CLIPModel.from_pretrained(CLIP_NAME, cache_dir=HF_HOME.as_posix()).to(DEVICE)
clip_model.eval()

@torch.no_grad()
def clip_text_emb(texts, batch_size=128, max_len=77, cache_path=None):
    if cache_path and Path(cache_path).exists():
        return np.load(cache_path)
    feats = []
    for i in trange(0, len(texts), batch_size, desc="CLIP text @ MPS"):
        batch = texts[i:i+batch_size]
        inputs = processor(
            text=batch, images=None, return_tensors="pt",
            padding=True, truncation=True, max_length=max_len
        )
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
        tf = clip_model.get_text_features(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
        tf = torch.nn.functional.normalize(tf, p=2, dim=1)
        feats.append(tf.to(torch.float32).cpu().numpy())
    arr = np.vstack(feats).astype(np.float32)
    if cache_path:
        np.save(cache_path, arr)
    return arr

@torch.no_grad()
def clip_image_emb_tta(paths, batch_size=64, cache_path=None):
    """
    Image TTA: embed original and horizontal-flipped images, then average.
    Falls back to a zero vector if an image path cannot be opened.
    """
    if cache_path and Path(cache_path).exists():
        return np.load(cache_path)

    feats, batch_o, batch_f = [], [], []

    def flush():
        nonlocal feats, batch_o, batch_f
        if not batch_o:
            return
        # Processor returns dict; we only need pixel_values
        inp1 = processor(images=batch_o, return_tensors="pt")["pixel_values"].to(DEVICE)
        inp2 = processor(images=batch_f, return_tensors="pt")["pixel_values"].to(DEVICE)

        f1 = clip_model.get_image_features(pixel_values=inp1)
        f2 = clip_model.get_image_features(pixel_values=inp2)

        f1 = torch.nn.functional.normalize(f1, p=2, dim=1)
        f2 = torch.nn.functional.normalize(f2, p=2, dim=1)

        f = ((f1 + f2) / 2.0).to(torch.float32).cpu().numpy()
        feats.append(f)

        batch_o, batch_f = [], []

    for p in tqdm(paths, desc="CLIP image TTA @ MPS"):
        try:
            img = Image.open(p).convert("RGB")
            img_flip = img.transpose(Image.FLIP_LEFT_RIGHT)
            batch_o.append(img)
            batch_f.append(img_flip)
        except Exception:
            # If image can't be read, append a single zero-vector with proper dim
            # Probe the output dim from the model’s visual projection
            z = np.zeros((1, clip_model.visual_projection.out_features), dtype=np.float32)
            feats.append(z)
            continue

        if len(batch_o) >= batch_size:
            flush()

    flush()
    arr = np.vstack(feats).astype(np.float32)
    if cache_path:
        np.save(cache_path, arr)
    return arr

# ------------- Caching paths -------------
CACHE_CLIP = Path("data/processed/improved_clip")
CACHE_CLIP.mkdir(parents=True, exist_ok=True)

print("Encoding text with CLIP...")
Ctxt_train  = clip_text_emb(train_texts,  cache_path=CACHE_CLIP/"clip_txt_train.npy")
Ctxt_test   = clip_text_emb(test_texts,   cache_path=CACHE_CLIP/"clip_txt_test.npy")
Ctxt_holdtr = clip_text_emb(holdtr_texts, cache_path=CACHE_CLIP/"clip_txt_holdtr.npy")
Ctxt_holdva = clip_text_emb(holdva_texts, cache_path=CACHE_CLIP/"clip_txt_holdva.npy")

print("Encoding images with CLIP (TTA: original + hflip)...")
Cimg_train  = clip_image_emb_tta(train_df["image_path"].tolist(), cache_path=CACHE_CLIP/"clip_img_train_tta.npy")
Cimg_test   = clip_image_emb_tta(test_df["image_path"].tolist(),  cache_path=CACHE_CLIP/"clip_img_test_tta.npy")
Cimg_holdtr = clip_image_emb_tta(hold_tr_df["image_path"].tolist(), cache_path=CACHE_CLIP/"clip_img_holdtr_tta.npy")
Cimg_holdva = clip_image_emb_tta(hold_va_df["image_path"].tolist(), cache_path=CACHE_CLIP/"clip_img_holdva_tta.npy")

print(f"\nCLIP embedding shapes:")
print(f"  Text:  {Ctxt_train.shape}")
print(f"  Image: {Cimg_train.shape} (TTA)")

# Cell 6: Improved Model Architectures + Training Functions

In [ ]:
# =========================
# Model Architectures
# =========================
class ImprovedRegressor(nn.Module):
    """4-layer MLP for text/image heads with BatchNorm"""
    def __init__(self, in_dim, hidden=512, drop=0.3):
        super().__init__()
        self.bn_input = nn.BatchNorm1d(in_dim)
        self.fc1 = nn.Linear(in_dim, hidden)
        self.bn1 = nn.BatchNorm1d(hidden)
        self.fc2 = nn.Linear(hidden, hidden // 2)
        self.bn2 = nn.BatchNorm1d(hidden // 2)
        self.fc3 = nn.Linear(hidden // 2, hidden // 4)
        self.bn3 = nn.BatchNorm1d(hidden // 4)
        self.fc4 = nn.Linear(hidden // 4, 1)
        self.dropout = nn.Dropout(drop)
        
    def forward(self, x):
        x = self.bn_input(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(self.bn1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(self.bn2(x))
        x = torch.relu(self.fc3(x))
        x = self.dropout(self.bn3(x))
        x = self.fc4(x)
        return x.squeeze(1)


class FusionRegressor(nn.Module):
    """5-layer deep fusion network for multimodal inputs"""
    def __init__(self, in_dim, hidden=768, drop=0.3):
        super().__init__()
        self.bn_input = nn.BatchNorm1d(in_dim)
        self.fc1 = nn.Linear(in_dim, hidden)
        self.bn1 = nn.BatchNorm1d(hidden)
        self.fc2 = nn.Linear(hidden, hidden // 2)
        self.bn2 = nn.BatchNorm1d(hidden // 2)
        self.fc3 = nn.Linear(hidden // 2, hidden // 4)
        self.bn3 = nn.BatchNorm1d(hidden // 4)
        self.fc4 = nn.Linear(hidden // 4, hidden // 8)
        self.bn4 = nn.BatchNorm1d(hidden // 8)
        self.fc5 = nn.Linear(hidden // 8, 1)
        self.dropout = nn.Dropout(drop)
        
    def forward(self, x):
        x = self.bn_input(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(self.bn1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(self.bn2(x))
        x = torch.relu(self.fc3(x))
        x = self.dropout(self.bn3(x))
        x = torch.relu(self.fc4(x))
        x = self.dropout(self.bn4(x))
        x = self.fc5(x)
        return x.squeeze(1)


# =========================
# SMAPE-based early stopping
# =========================
class QuantileLoss(nn.Module):
    """
    Pinball loss (quantile regression). q=0.5 is median, robust for heavy-tailed prices.
    Use instead of Huber if your targets are spiky.
    """
    def __init__(self, q=0.5):
        super().__init__()
        self.q = q
    def forward(self, pred, target):
        # pred/target are in log-space here
        e = target - pred
        return torch.max(self.q * e, (self.q - 1) * e).mean()

@torch.no_grad()
def _val_smape_price(model, Xva, yva_log):
    """
    Compute validation SMAPE in price space.
    yva_log is log1p(price); convert both pred and target to price via expm1.
    """
    model.eval()
    p_va_log = model(Xva)
    p_va_price = torch.expm1(p_va_log).clamp_min(0).cpu().numpy()
    y_va_price = torch.expm1(yva_log).clamp_min(0).cpu().numpy()
    return smape_np(y_va_price, p_va_price)

def train_head(
    Xtr, ytr_log, Xva, yva_log, DEVICE,
    epochs=25, bs=512, lr=1e-3, wd=1e-3,
    hidden=512, drop=0.3, patience=7,
    use_quantile=True, huber_delta=1.0
):
    """
    Train MLP head (text) in log-price space; early-stop on validation SMAPE (price space).
    """
    model = ImprovedRegressor(Xtr.shape[1], hidden=hidden, drop=drop).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)

    loss_fn = QuantileLoss(q=0.5) if use_quantile else nn.HuberLoss(delta=huber_delta)

    best_smape = float("inf")
    best_state = None
    bad_epochs = 0

    for ep in range(epochs):
        model.train()
        idx = torch.randperm(Xtr.shape[0], device=DEVICE)
        for i in range(0, len(idx), bs):
            b = idx[i:i+bs]
            pred = model(Xtr[b])
            loss = loss_fn(pred, ytr_log[b])
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
        sched.step()

        # --- Early stop on SMAPE (price space) ---
        sm = _val_smape_price(model, Xva, yva_log)
        if sm < best_smape:
            best_smape = sm
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    model.eval()
    with torch.no_grad():
        p_va = model(Xva)  # still log-space; caller will expm1 when needed
    return model, p_va

def train_fusion(
    Xtr, ytr_log, Xva, yva_log, DEVICE,
    epochs=30, bs=512, lr=5e-4, wd=1e-3,
    hidden=768, drop=0.3, patience=8,
    use_quantile=True, huber_delta=1.0
):
    """
    Train fusion MLP (text+image+features) in log-price; early-stop on SMAPE.
    """
    model = FusionRegressor(Xtr.shape[1], hidden=hidden, drop=drop).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)

    loss_fn = QuantileLoss(q=0.5) if use_quantile else nn.HuberLoss(delta=huber_delta)

    best_smape = float("inf")
    best_state = None
    bad_epochs = 0

    for ep in range(epochs):
        model.train()
        idx = torch.randperm(Xtr.shape[0], device=DEVICE)
        for i in range(0, len(idx), bs):
            b = idx[i:i+bs]
            pred = model(Xtr[b])
            loss = loss_fn(pred, ytr_log[b])
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
        sched.step()

        # --- Early stop on SMAPE (price space) ---
        sm = _val_smape_price(model, Xva, yva_log)
        if sm < best_smape:
            best_smape = sm
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    model.eval()
    with torch.no_grad():
        p_va = model(Xva)  # log-space
    return model, p_va

# Cell 7: Train Text Head (5-Fold CV)

In [ ]:
from sklearn.preprocessing import StandardScaler

print("\n" + "="*60)
print("Training Text Head (MiniLM + Features)  [fold-local scaling]")
print("="*60)

oof_txt = np.zeros(len(train_df), dtype=np.float32)
test_txt_accum = np.zeros(len(test_df), dtype=np.float32)

for fold in range(5):
    print(f"\nFold {fold+1}/5...")
    tr = train_df[train_df.fold != fold]
    va = train_df[train_df.fold == fold]

    # --- Fold-local scaler: fit ONLY on TRAIN rows of this fold ---
    scaler_f = StandardScaler()
    feats_tr = scaler_f.fit_transform(train_df.loc[tr.index, feature_cols])
    feats_va = scaler_f.transform(train_df.loc[va.index, feature_cols])
    feats_te = scaler_f.transform(test_df[feature_cols])

    # We concatenate embeddings (from MiniLM) with the per-fold scaled features
    # NOTE: use *raw* text embeddings here (Etxt_*_emb), not the globally-concatenated Etxt_train
    Xtr_np = np.hstack([Etxt_train_emb[tr.index], feats_tr]).astype(np.float32)
    Xva_np = np.hstack([Etxt_train_emb[va.index], feats_va]).astype(np.float32)
    Xte_np = np.hstack([Etxt_test_emb,            feats_te]).astype(np.float32)

    # To device
    Xtr = torch.from_numpy(Xtr_np).to(DEVICE)
    Xva = torch.from_numpy(Xva_np).to(DEVICE)
    Xte = torch.from_numpy(Xte_np).to(DEVICE)

    ytr = torch.from_numpy(np.log1p(tr.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)
    yva = torch.from_numpy(np.log1p(va.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)

    # Train (uses your train_head; consider SMAPE-based ES from earlier suggestions)
    model, p_va = train_head(
        Xtr, ytr, Xva, yva, DEVICE,
        epochs=25, bs=512, lr=1e-3, wd=1e-3,
        hidden=512, drop=0.3
    )

    with torch.no_grad():
        p_te = model(Xte)

    oof_txt[va.index] = torch.expm1(p_va).cpu().numpy()
    test_txt_accum += torch.expm1(p_te).cpu().numpy()

sm_txt = smape_np(train_df.price.values, np.clip(oof_txt, 0.0, None))
print(f"\n{'='*60}")
print(f"Text Head OOF SMAPE: {sm_txt:.4f}")
print(f"{'='*60}")

np.save(OUT_DIR/"oof"/"txt_improved.npy", oof_txt)
pd.DataFrame({
    "sample_id": test_df.sample_id,
    "price": np.clip(test_txt_accum/5.0, 0.0, None)
}).to_csv(OUT_DIR/"test_preds"/"txt_improved.csv", index=False)

# ----------------------------
# Holdout evaluation (no leak)
# ----------------------------
print("\nHoldout evaluation (fold-local style for holdout)...")

# Fit a brand-new scaler ONLY on holdout-train
scaler_h = StandardScaler()
fe_htr = scaler_h.fit_transform(hold_tr_df[feature_cols])
fe_hva = scaler_h.transform(hold_va_df[feature_cols])

Xtr_h = torch.from_numpy(np.hstack([Etxt_holdtr_emb, fe_htr]).astype(np.float32)).to(DEVICE)
Xva_h = torch.from_numpy(np.hstack([Etxt_holdva_emb, fe_hva]).astype(np.float32)).to(DEVICE)

ytr_h = torch.from_numpy(np.log1p(hold_tr_df.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)
yva_h = torch.from_numpy(np.log1p(hold_va_df.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)

model_txt_hold, p_va_h = train_head(
    Xtr_h, ytr_h, Xva_h, yva_h, DEVICE,
    epochs=25, bs=512, lr=1e-3, wd=1e-3
)
pred_hold_txt = torch.expm1(p_va_h).cpu().numpy()
sm_txt_hold = smape_np(hold_va_df.price.values, np.clip(pred_hold_txt, 0.0, None))
print(f"Text Head Holdout SMAPE: {sm_txt_hold:.4f}")

# Cell 8: Train Image Head (5-Fold CV)

In [ ]:
print("\n" + "="*60)
print("Training Image Head (CLIP Image)")
print("="*60)

oof_img = np.zeros(len(train_df), dtype=np.float32)
test_img_accum = np.zeros(len(test_df), dtype=np.float32)

for fold in range(5):
    print(f"\nFold {fold+1}/5...")
    tr = train_df[train_df.fold != fold]
    va = train_df[train_df.fold == fold]

    # Convert to float32 explicitly before creating tensors (MPS doesn't support float64)
    Xtr = torch.from_numpy(Cimg_train[tr.index].astype(np.float32)).to(DEVICE)
    Xva = torch.from_numpy(Cimg_train[va.index].astype(np.float32)).to(DEVICE)
    Xte = torch.from_numpy(Cimg_test.astype(np.float32)).to(DEVICE)

    ytr = torch.from_numpy(np.log1p(tr.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)
    yva = torch.from_numpy(np.log1p(va.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)

    model, p_va = train_head(Xtr, ytr, Xva, yva, DEVICE, epochs=25, bs=512, lr=8e-4,
                             wd=1e-3, hidden=512, drop=0.3)
    
    with torch.no_grad():
        p_te = model(Xte)

    oof_img[va.index] = torch.expm1(p_va).cpu().numpy()
    test_img_accum += torch.expm1(p_te).cpu().numpy()

sm_img = smape_np(train_df.price.values, np.clip(oof_img, 0.0, None))
print(f"\n{'='*60}")
print(f"Image Head OOF SMAPE: {sm_img:.4f}")
print(f"{'='*60}")

np.save(OUT_DIR/"oof"/"img_improved.npy", oof_img)
pd.DataFrame({
    "sample_id": test_df.sample_id,
    "price": np.clip(test_img_accum/5.0, 0.0, None)
}).to_csv(OUT_DIR/"test_preds"/"img_improved.csv", index=False)

# Holdout evaluation
print("\nHoldout evaluation...")
Xtr = torch.from_numpy(Cimg_holdtr.astype(np.float32)).to(DEVICE)
Xva = torch.from_numpy(Cimg_holdva.astype(np.float32)).to(DEVICE)
ytr = torch.from_numpy(np.log1p(hold_tr_df.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)
yva = torch.from_numpy(np.log1p(hold_va_df.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)

model_img_hold, p_va = train_head(Xtr, ytr, Xva, yva, DEVICE, epochs=25, bs=512, lr=8e-4, wd=1e-3)
pred_hold_img = torch.expm1(p_va).cpu().numpy()
sm_img_hold = smape_np(hold_va_df.price.values, np.clip(pred_hold_img, 0.0, None))
print(f"Image Head Holdout SMAPE: {sm_img_hold:.4f}")

# Cell 9: Train Multimodal Fusion (5-Fold CV)

In [ ]:
from sklearn.preprocessing import StandardScaler

print("\n" + "="*60)
print("Training Multimodal Fusion (CLIP Text + Image + Features)  [fold-local scaling]")
print("="*60)

oof_mm = np.zeros(len(train_df), dtype=np.float32)
test_mm_accum = np.zeros(len(test_df), dtype=np.float32)

# We will build Xtr/Xva/Xte inside each fold with a fold-local scaler
# so we don't precompute mm_train/mm_test with globally-scaled features.

for fold in range(5):
    print(f"\nFold {fold+1}/5...")
    tr = train_df[train_df.fold != fold]
    va = train_df[train_df.fold == fold]

    # --- Fold-local scaler on handcrafted features ---
    scaler_f = StandardScaler()
    feats_tr = scaler_f.fit_transform(train_df.loc[tr.index, feature_cols])
    feats_va = scaler_f.transform(train_df.loc[va.index, feature_cols])
    feats_te = scaler_f.transform(test_df[feature_cols])

    # Concatenate CLIP text + CLIP image + per-fold scaled features
    Xtr_np = np.hstack([Ctxt_train[tr.index], Cimg_train[tr.index], feats_tr]).astype(np.float32)
    Xva_np = np.hstack([Ctxt_train[va.index], Cimg_train[va.index], feats_va]).astype(np.float32)
    Xte_np = np.hstack([Ctxt_test,            Cimg_test,            feats_te]).astype(np.float32)

    Xtr = torch.from_numpy(Xtr_np).to(DEVICE)
    Xva = torch.from_numpy(Xva_np).to(DEVICE)
    Xte = torch.from_numpy(Xte_np).to(DEVICE)

    ytr = torch.from_numpy(np.log1p(tr.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)
    yva = torch.from_numpy(np.log1p(va.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)

    model, p_va = train_fusion(
        Xtr, ytr, Xva, yva, DEVICE,
        epochs=30, bs=512, lr=5e-4, wd=1e-3,
        hidden=768, drop=0.3
    )

    with torch.no_grad():
        p_te = model(Xte)

    oof_mm[va.index] = torch.expm1(p_va).cpu().numpy()
    test_mm_accum += torch.expm1(p_te).cpu().numpy()

sm_mm = smape_np(train_df.price.values, np.clip(oof_mm, 0.0, None))
print(f"\n{'='*60}")
print(f"Multimodal Fusion OOF SMAPE: {sm_mm:.4f}")
print(f"{'='*60}")

np.save(OUT_DIR/"oof"/"mm_improved.npy", oof_mm)
pd.DataFrame({
    "sample_id": test_df.sample_id,
    "price": np.clip(test_mm_accum/5.0, 0.0, None)
}).to_csv(OUT_DIR/"test_preds"/"mm_improved.csv", index=False)

# ----------------------------
# Holdout evaluation (no leak)
# ----------------------------
print("\nHoldout evaluation (fold-local style for holdout)...")

scaler_h = StandardScaler()
fe_htr = scaler_h.fit_transform(hold_tr_df[feature_cols])
fe_hva = scaler_h.transform(hold_va_df[feature_cols])

Xtr_h = torch.from_numpy(np.hstack([Ctxt_holdtr, Cimg_holdtr, fe_htr]).astype(np.float32)).to(DEVICE)
Xva_h = torch.from_numpy(np.hstack([Ctxt_holdva, Cimg_holdva, fe_hva]).astype(np.float32)).to(DEVICE)

ytr_h = torch.from_numpy(np.log1p(hold_tr_df.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)
yva_h = torch.from_numpy(np.log1p(hold_va_df.price.values.clip(min=0.0)).astype(np.float32)).to(DEVICE)

model_mm_hold, p_va_h = train_fusion(
    Xtr_h, ytr_h, Xva_h, yva_h, DEVICE,
    epochs=30, bs=512, lr=5e-4, wd=1e-3
)
pred_hold_mm = torch.expm1(p_va_h).cpu().numpy()
sm_mm_hold = smape_np(hold_va_df.price.values, np.clip(pred_hold_mm, 0.0, None))
print(f"Multimodal Fusion Holdout SMAPE: {sm_mm_hold:.4f}")

# Cell 10: Train Gradient Boosting Models (XGBoost + LightGBM)

In [ ]:
# ============================================================
# Cell 10 (REPLACED): Gradient Boosting Models (XGB, LGB, LGB-Tweedie)
# No scaling (trees don't need it) + add Tweedie on raw price
# ============================================================

print("\n" + "="*60)
print("Training Gradient Boosting Models")
print("="*60)

# Install if needed
try:
    import xgboost as xgb
    import lightgbm as lgb
except ImportError:
    print("Installing XGBoost and LightGBM...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost", "lightgbm", "-q"])
    import xgboost as xgb
    import lightgbm as lgb

# ------------------------------------------------------------------
# Build GBM feature matrices: [MiniLM txt emb | CLIP txt | CLIP img | raw handcrafted]
# We AVOID global scaling here to prevent CV leakage (trees don't need scaling).
# ------------------------------------------------------------------
dense_train = train_df[feature_cols].astype(np.float32).values
dense_test  = test_df[feature_cols].astype(np.float32).values
dense_htr   = hold_tr_df[feature_cols].astype(np.float32).values
dense_hva   = hold_va_df[feature_cols].astype(np.float32).values

X_full_train  = np.hstack([Etxt_train_emb,  Ctxt_train,  Cimg_train,  dense_train]).astype(np.float32)
X_full_test   = np.hstack([Etxt_test_emb,   Ctxt_test,   Cimg_test,   dense_test]).astype(np.float32)
X_full_holdtr = np.hstack([Etxt_holdtr_emb, Ctxt_holdtr, Cimg_holdtr, dense_htr]).astype(np.float32)
X_full_holdva = np.hstack([Etxt_holdva_emb, Ctxt_holdva, Cimg_holdva, dense_hva]).astype(np.float32)

print(f"Full GBM feature dimension: {X_full_train.shape[1]}")

# ============================================================
# 5-fold XGBoost (log-price) ---------------------------------
# ============================================================
print("\nTraining XGBoost...")
oof_xgb = np.zeros(len(train_df), dtype=np.float32)
test_xgb_accum = np.zeros(len(test_df), dtype=np.float32)

for fold in range(5):
    print(f"  Fold {fold+1}/5...", end=" ")
    tr = train_df[train_df.fold != fold]
    va = train_df[train_df.fold == fold]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        learning_rate=0.03,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=3,
        gamma=0.1,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=SEED + fold,
        tree_method='hist',
        n_jobs=-1
    )

    model.fit(
        X_full_train[tr.index],
        np.log1p(tr.price.values),
        eval_set=[(X_full_train[va.index], np.log1p(va.price.values))],
        verbose=False
    )

    oof_xgb[va.index]   = np.expm1(model.predict(X_full_train[va.index]))
    test_xgb_accum     += np.expm1(model.predict(X_full_test))
    print("Done")

sm_xgb = smape_np(train_df.price.values, np.clip(oof_xgb, 0.0, None))
print(f"\nXGBoost OOF SMAPE: {sm_xgb:.4f}")

# ============================================================
# 5-fold LightGBM (log-price) --------------------------------
# ============================================================
print("\nTraining LightGBM...")
oof_lgb = np.zeros(len(train_df), dtype=np.float32)
test_lgb_accum = np.zeros(len(test_df), dtype=np.float32)

for fold in range(5):
    print(f"  Fold {fold+1}/5...", end=" ")
    tr = train_df[train_df.fold != fold]
    va = train_df[train_df.fold == fold]

    model = lgb.LGBMRegressor(
        n_estimators=2000,
        learning_rate=0.03,
        max_depth=-1,           # let leaves control depth
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_samples=20,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=SEED + fold,
        n_jobs=-1,
        verbose=-1
    )

    model.fit(
        X_full_train[tr.index],
        np.log1p(tr.price.values),
        eval_set=[(X_full_train[va.index], np.log1p(va.price.values))],
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )

    oof_lgb[va.index]   = np.expm1(model.predict(X_full_train[va.index]))
    test_lgb_accum     += np.expm1(model.predict(X_full_test))
    print(f"Done (trees: {model.best_iteration_})")

sm_lgb = smape_np(train_df.price.values, np.clip(oof_lgb, 0.0, None))
print(f"\nLightGBM OOF SMAPE: {sm_lgb:.4f}")

# ============================================================
# 5-fold LightGBM Tweedie (RAW price) ------------------------
# ============================================================
print("\nTraining LightGBM Tweedie...")
oof_lgb_twd = np.zeros(len(train_df), dtype=np.float32)
test_lgb_twd_acc = np.zeros(len(test_df), dtype=np.float32)

for fold in range(5):
    print(f"  Fold {fold+1}/5...", end=" ")
    tr = train_df[train_df.fold != fold]
    va = train_df[train_df.fold == fold]

    lgb_tweedie = lgb.LGBMRegressor(
        objective="tweedie",
        tweedie_variance_power=1.2,   # try 1.1–1.6 if you tune
        n_estimators=4000,
        learning_rate=0.02,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=SEED + fold,
        n_jobs=-1,
        verbose=-1
    )

    # Train on RAW price
    lgb_tweedie.fit(
        X_full_train[tr.index], train_df.loc[tr.index, "price"].values,
        eval_set=[(X_full_train[va.index], train_df.loc[va.index, "price"].values)],
        callbacks=[lgb.early_stopping(200, verbose=False)]
    )

    oof_lgb_twd[va.index]   = np.clip(lgb_tweedie.predict(X_full_train[va.index]), 0.0, None)
    test_lgb_twd_acc       += np.clip(lgb_tweedie.predict(X_full_test),            0.0, None)
    print("Done")

sm_lgb_twd = smape_np(train_df.price.values, np.clip(oof_lgb_twd, 0.0, None))
print(f"\nLightGBM Tweedie OOF SMAPE: {sm_lgb_twd:.4f}")

# ------------------------------------------------------------
# Save GBM OOFs + Test predictions
# ------------------------------------------------------------
np.save(OUT_DIR/"oof"/"xgb.npy",         oof_xgb)
np.save(OUT_DIR/"oof"/"lgb.npy",         oof_lgb)
np.save(OUT_DIR/"oof"/"lgb_tweedie.npy", oof_lgb_twd)

pd.DataFrame({"sample_id": test_df.sample_id, "price": np.clip(test_xgb_accum/5.0,      0.0, None)}).to_csv(OUT_DIR/"test_preds"/"xgb.csv",          index=False)
pd.DataFrame({"sample_id": test_df.sample_id, "price": np.clip(test_lgb_accum/5.0,      0.0, None)}).to_csv(OUT_DIR/"test_preds"/"lgb.csv",          index=False)
pd.DataFrame({"sample_id": test_df.sample_id, "price": np.clip(test_lgb_twd_acc/5.0,    0.0, None)}).to_csv(OUT_DIR/"test_preds"/"lgb_tweedie.csv",  index=False)

# ------------------------------------------------------------
# Holdout evaluation for all three
# ------------------------------------------------------------
print("\nHoldout evaluation for GBMs...")

# XGBoost (log-price)
xgb_hold = xgb.XGBRegressor(
    n_estimators=2000, learning_rate=0.03, max_depth=8,
    subsample=0.8, colsample_bytree=0.8, random_state=SEED,
    tree_method='hist', n_jobs=-1
)
xgb_hold.fit(
    X_full_holdtr, np.log1p(hold_tr_df.price.values),
    eval_set=[(X_full_holdva, np.log1p(hold_va_df.price.values))],
    verbose=False
)
pred_hold_xgb = np.expm1(xgb_hold.predict(X_full_holdva))
sm_xgb_hold = smape_np(hold_va_df.price.values, np.clip(pred_hold_xgb, 0.0, None))

# LightGBM (log-price)
lgb_hold = lgb.LGBMRegressor(
    n_estimators=2000, learning_rate=0.03, num_leaves=63,
    subsample=0.8, colsample_bytree=0.8, random_state=SEED, verbose=-1
)
lgb_hold.fit(
    X_full_holdtr, np.log1p(hold_tr_df.price.values),
    eval_set=[(X_full_holdva, np.log1p(hold_va_df.price.values))],
    callbacks=[lgb.early_stopping(100, verbose=False)]
)
pred_hold_lgb = np.expm1(lgb_hold.predict(X_full_holdva))
sm_lgb_hold = smape_np(hold_va_df.price.values, np.clip(pred_hold_lgb, 0.0, None))

# LightGBM Tweedie (RAW price)
lgb_twd_hold = lgb.LGBMRegressor(
    objective="tweedie",
    tweedie_variance_power=1.2,
    n_estimators=4000, learning_rate=0.02, num_leaves=63,
    subsample=0.8, colsample_bytree=0.8, random_state=SEED, verbose=-1
)
lgb_twd_hold.fit(
    X_full_holdtr, hold_tr_df.price.values,
    eval_set=[(X_full_holdva, hold_va_df.price.values)],
    callbacks=[lgb.early_stopping(200, verbose=False)]
)
pred_hold_lgb_twd = np.clip(lgb_twd_hold.predict(X_full_holdva), 0.0, None)
sm_lgb_twd_hold = smape_np(hold_va_df.price.values, pred_hold_lgb_twd)

print(f"XGBoost Holdout SMAPE:        {sm_xgb_hold:.4f}")
print(f"LightGBM (log) Holdout SMAPE: {sm_lgb_hold:.4f}")
print(f"LightGBM Tweedie Holdout:     {sm_lgb_twd_hold:.4f}")
print("="*60)

# Cell 11: Ensemble Optimization

In [ ]:
# ============================================================
# Cell 11 (REPLACED): Ensemble Optimization + Local Refinement
# - Coarse grid search (step=0.05)
# - Local refinement around the best (±0.10, step=0.01)
# - Auto-include lgb_tweedie and stack_ridge if available
# ============================================================

print("\n" + "="*60)
print("Ensemble Optimization")
print("="*60)

# ------------------------------------------------------------
# Collect OOF predictions
# ------------------------------------------------------------
oofs = {
    "text": oof_txt,
    "img":  oof_img,
    "mm":   oof_mm,
    "xgb":  oof_xgb,
    "lgb":  oof_lgb,
}

# Optionally include LightGBM Tweedie if trained in Cell 10
if "oof_lgb_twd" in globals():
    oofs["lgb_tweedie"] = oof_lgb_twd

# Optionally include Ridge stacker if you added it
# (variables typically named oof_stack and/or saved test CSV in Cell "stacker")
if "oof_stack" in globals():
    oofs["stack_ridge"] = oof_stack

# Ground-truth
y = train_df["price"].values

def grid_weights(keys, step=0.05):
    """Generate all weight combinations that sum to 1 with given step."""
    n = len(keys)
    w = np.arange(0, 1 + 1e-9, step)

    if n == 1:
        yield {keys[0]: 1.0}
        return

    def rec(prefix, remain, total):
        if remain == 1:
            yield prefix + [total]
            return
        for a in w:
            if a > total:
                break
            yield from rec(prefix + [a], remain - 1, total - a)

    for weights in rec([], n, 1.0):
        yield dict(zip(keys, weights))

keys = list(oofs.keys())
X = np.stack([oofs[k] for k in keys], axis=1)

print(f"\nSearching {len(keys)}-model ensemble space (coarse step=0.05)...")
best = (1e9, None)
tested = 0

for wmap in grid_weights(keys, step=0.05):
    w = np.array([wmap[k] for k in keys], dtype=np.float32)
    pred = (X * w[None, :]).sum(axis=1)
    sm = smape_np(y, np.clip(pred, 0.0, None))
    if sm < best[0]:
        best = (sm, wmap)
    tested += 1
    if tested % 10000 == 0:
        print(f"  Tested {tested} combos, best so far: {best[0]:.4f}")

best_oof_smape, best_w = best
print(f"\n{'='*60}")
print(f"Best (coarse) Ensemble OOF SMAPE: {best_oof_smape:.4f}")
print("Coarse optimal weights:")
for k, v in best_w.items():
    print(f"  {k:12s}: {v:.3f}")
print("="*60)

# ------------------------------------------------------------
# Local refinement around the best (±0.10, step 0.01)
# ------------------------------------------------------------
print("\nRefining around the coarse optimum (±0.10, step=0.01)...")
base = np.array([best_w[k] for k in keys], dtype=np.float32)

def local_refine(base, radius=0.10, step=0.01):
    grid = np.arange(-radius, radius + 1e-9, step)
    best = (1e9, base.copy())
    # To keep the loops tractable, adjust first min(3, len(base)) weights locally.
    k = min(3, len(base))
    for d0 in grid:
        for d1 in grid:
            # if k==2, skip d2 loop; if k>=3, include it
            if k >= 3:
                ds = (d0, d1)
                for d2 in grid:
                    d = base.copy()
                    d[:3] += np.array([d0, d1, d2])
                    d = np.clip(d, 0, None)
                    s = d.sum()
                    if s <= 0: 
                        continue
                    d /= s
                    sm = smape_np(y, np.clip((X * d[None, :]).sum(axis=1), 0.0, None))
                    if sm < best[0]:
                        best = (sm, d.copy())
            else:
                d = base.copy()
                d[:2] += np.array([d0, d1])
                d = np.clip(d, 0, None)
                s = d.sum()
                if s <= 0: 
                    continue
                d /= s
                sm = smape_np(y, np.clip((X * d[None, :]).sum(axis=1), 0.0, None))
                if sm < best[0]:
                    best = (sm, d.copy())
    return best

ref_sm, ref_w = local_refine(base, radius=0.10, step=0.01)
best_oof_smape = float(ref_sm)
best_w = dict(zip(keys, ref_w.tolist()))

print(f"Refined OOF SMAPE: {best_oof_smape:.4f}")
print("Refined weights:")
for k in keys:
    print(f"  {k:12s}: {best_w[k]:.3f}")

# ------------------------------------------------------------
# Holdout ensemble using the refined weights
# ------------------------------------------------------------
hold_pred_map = {}

# Mandatory holdout preds (from Cells 7–10)
if "pred_hold_txt" in globals(): hold_pred_map["text"] = pred_hold_txt
if "pred_hold_img" in globals(): hold_pred_map["img"]  = pred_hold_img
if "pred_hold_mm" in globals():  hold_pred_map["mm"]   = pred_hold_mm
if "pred_hold_xgb" in globals(): hold_pred_map["xgb"]  = pred_hold_xgb
if "pred_hold_lgb" in globals(): hold_pred_map["lgb"]  = pred_hold_lgb

# Optional additions (if present)
if "pred_hold_lgb_twd" in globals():
    hold_pred_map["lgb_tweedie"] = pred_hold_lgb_twd
if "pred_hold_stack" in globals():
    hold_pred_map["stack_ridge"] = pred_hold_stack

# Ensure we only ensemble keys for which we have BOTH OOF and holdout
keys = [k for k in keys if k in hold_pred_map]
w = np.array([best_w[k] for k in keys], dtype=np.float32)
hold_stack = np.stack([hold_pred_map[k] for k in keys], axis=1)
hold_pred = (hold_stack * w[None, :]).sum(axis=1)
sm_hold_ens = smape_np(hold_va_df.price.values, np.clip(hold_pred, 0.0, None))

print(f"\nEnsemble Holdout SMAPE (refined): {sm_hold_ens:.4f}")
print("="*60)

# ------------------------------------------------------------
# Save ensemble report (keys + weights + scores)
# ------------------------------------------------------------
report = {
    "oof_smape": float(best_oof_smape),
    "holdout_smape": float(sm_hold_ens),
    "keys": keys,  # Order matters for Cell 12 test blending
    "weights": [float(best_w[k]) for k in keys],
    "weights_map": {k: float(best_w[k]) for k in keys},
    "individual_oof_scores": {
        "text": float(sm_txt) if "sm_txt" in globals() else None,
        "img":  float(sm_img) if "sm_img" in globals() else None,
        "mm":   float(sm_mm)  if "sm_mm"  in globals() else None,
        "xgb":  float(sm_xgb) if "sm_xgb" in globals() else None,
        "lgb":  float(sm_lgb) if "sm_lgb" in globals() else None,
        "lgb_tweedie": float(sm_lgb_twd) if "sm_lgb_twd" in globals() else None,
        "stack_ridge": float(sm_stack)   if "sm_stack"   in globals() else None,
    }
}
(OUT_DIR/"reports").mkdir(parents=True, exist_ok=True)
with open(OUT_DIR/"reports"/"ensemble_improved.json", "w") as f:
    json.dump(report, f, indent=2)

print("Saved refined ensemble report to", (OUT_DIR/"reports"/"ensemble_improved.json").as_posix())

In [ ]:
from sklearn.linear_model import Ridge

stack_names = ["text","img","mm","xgb","lgb"]
# add if present:
if (OUT_DIR/"oof"/"lgb_tweedie.npy").exists():
    stack_names.append("lgb_tweedie")

X_stack = np.stack([np.log1p(np.clip(oofs[n], 0, 1e6)) for n in stack_names], axis=1)
y_stack = np.log1p(np.clip(train_df["price"].values, 0, 1e6))

ridge = Ridge(alpha=1.0, random_state=SEED)
ridge.fit(X_stack, y_stack)

# Build test stack in same order
def load_test_csv(name):
    # Map model names to actual CSV filenames
    file_map = {
        "text": "txt_improved",
        "img": "img_improved",
        "mm": "mm_improved",
        "xgb": "xgb",
        "lgb": "lgb",
        "lgb_tweedie": "lgb_tweedie"
    }
    filename = file_map.get(name, name)
    preds = pd.read_csv(OUT_DIR/"test_preds"/f"{filename}.csv")["price"].values
    return np.clip(preds, 0, 1e6)  # Clip to avoid inf in log1p
T_stack = np.stack([np.log1p(load_test_csv(n)) for n in stack_names], axis=1)

oof_stack = np.expm1(ridge.predict(X_stack))
sm_stack = smape_np(train_df["price"].values, np.clip(oof_stack, 0.0, None))
print("Ridge stacker OOF SMAPE:", sm_stack)

test_stack = np.expm1(ridge.predict(T_stack))
pd.DataFrame({"sample_id": test_df.sample_id, "price": np.clip(test_stack, 0.0, None)}).to_csv(OUT_DIR/"test_preds"/"stack_ridge.csv", index=False)

# add to ensemble pool
oofs["stack_ridge"] = oof_stack


# Cell 12: Generate Final Predictions + Post-Processing

In [ ]:
print("\n" + "="*60)
print("Generating Final Ensemble Predictions")
print("="*60)

# Load individual test predictions
t_text = pd.read_csv(OUT_DIR/"test_preds"/"txt_improved.csv")
t_img = pd.read_csv(OUT_DIR/"test_preds"/"img_improved.csv")
t_mm = pd.read_csv(OUT_DIR/"test_preds"/"mm_improved.csv")
t_xgb = pd.read_csv(OUT_DIR/"test_preds"/"xgb.csv")
t_lgb = pd.read_csv(OUT_DIR/"test_preds"/"lgb.csv")

# Merge all predictions
sub = t_text.rename(columns={"price": "price_text"}) \
    .merge(t_img.rename(columns={"price": "price_img"}), on="sample_id") \
    .merge(t_mm.rename(columns={"price": "price_mm"}), on="sample_id") \
    .merge(t_xgb.rename(columns={"price": "price_xgb"}), on="sample_id") \
    .merge(t_lgb.rename(columns={"price": "price_lgb"}), on="sample_id")

# Apply ensemble weights
weights = np.array([best_w[k] for k in ["text", "img", "mm", "xgb", "lgb"]], dtype=np.float32)
sub["price"] = sub[["price_text", "price_img", "price_mm", "price_xgb", "price_lgb"]].values.dot(weights)

# Post-processing: clip to training distribution bounds
print("\nApplying post-processing...")
lower_bound = train_df['price'].quantile(0.001)
upper_bound = train_df['price'].quantile(0.999)
print(f"  Clipping to [{lower_bound:.2f}, {upper_bound:.2f}]")

sub["price"] = sub["price"].clip(lower_bound, upper_bound)

# Final submission
final = sub[["sample_id", "price"]].copy()
final.to_csv(OUT_DIR/"test_preds"/"ensemble_improved.csv", index=False)

print(f"\n✅ Final submission saved to: {(OUT_DIR/'test_preds'/'ensemble_improved.csv').as_posix()}")
print(f"\nSubmission preview:")
print(final.head(10))

print("\n" + "="*60)
print("🎉 IMPROVED PIPELINE COMPLETE!")
print("="*60)
print(f"\n📊 Final Results Summary:")
print(f"\nIndividual Model OOF SMAPE:")
print(f"  Text Head:          {sm_txt:.4f}")
print(f"  Image Head:         {sm_img:.4f}")
print(f"  Multimodal Fusion:  {sm_mm:.4f}")
print(f"  XGBoost:            {sm_xgb:.4f}")
print(f"  LightGBM:           {sm_lgb:.4f}")
print(f"\n🏆 Ensemble OOF SMAPE:     {best_oof_smape:.4f}")
print(f"🎯 Ensemble Holdout SMAPE: {sm_hold_ens:.4f}")
print(f"\nEnsemble Weights:")
for k, v in best_w.items():
    print(f"  {k:15s}: {v:.3f}")
print("="*60)

# Check if target achieved
if sm_hold_ens < 40:
    print(f"\n🎉 TARGET ACHIEVED! Holdout SMAPE = {sm_hold_ens:.4f} < 40")
else:
    print(f"\n⚠️  Target not met. Holdout SMAPE = {sm_hold_ens:.4f}")
    print("\nSuggestions for further improvement:")
    print("  1. Use larger CLIP model (ViT-L/14 instead of ViT-B/32)")
    print("  2. Add TF-IDF features from text")
    print("  3. Implement pseudo-labeling on test set")
    print("  4. Add more domain-specific features (brand recognition, category)")
    print("  5. Use CatBoost in addition to XGBoost/LightGBM")
    print("  6. Increase training epochs with better early stopping")